In [1]:
#hide
#skip
%config Completer.use_jedi = False
# upgrade fastrl on colab
! [ -e /content ] && pip install -Uqq fastrl['dev'] pyvirtualdisplay && \
                     apt-get install -y xvfb python-opengl > /dev/null 2>&1 
# NOTE: IF YOU SEE VERSION ERRORS, IT IS SAFE TO IGNORE THEM. COLAB IS BEHIND IN SOME OF THE PACKAGE VERSIONS

In [2]:
# hide
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbverbose.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON
else:
    # Virutual display is needed for colab
    from pyvirtualdisplay import Display
    display=Display(visible=0,size=(400,300))
    display.start()

In [3]:
# default_exp fastai.loop

In [4]:
# export
# Python native modules
import os
from copy import deepcopy
from typing import *
import types
import logging
import inspect
# Third party libs
from fastcore.all import *
# Local modules

_logger=logging.getLogger(__name__)

# Loop
> fastrl concept of generic loop objects. 

The goal for Loops is to make it easy to customize, and know how sections of code connects
to other parts.

### Why do we need this?
We have identified at least 3 different kinds of loops already:

    Learner (training)
    Source/Gym (Data Access)
    Agent (How an AI takes in data, generates actions)

### What is a loop?

    It should be capable of containing inner loops. 
    It should be able to handle "phases" that might be similar to each other. 
    It should self-describe its structure. 
    It should be easy to know which parts of the loop are taking long/short amounts of time.
    It should be flexible in state modification.
    It should alternatively make it easy show what fields are being changed at what points in time.

A Loop will act as a compiled structure. The actual result will be a compiled list of nodes that reference the original loop.

Loop is a compiled object that organizes the callbacks and loop calls into a possibly repeating sequence.

`Literal['order',int]` contains a int that should be more than or equal to 0. This determines when a 
function in the loops should be executed relative to other functions.

In [75]:
# export
PREFIXES=['before_','on_','after_','failed_','finally_']

class NodeException(Exception):pass

class Node(object):
    def __init__(self,
                 function:Callable,
                 parent:Optional['Node']=None,
                 children:Optional[List['Node']]=None):
        store_attr()
        self.order=None
        for anno in L(anno_ret(self.function)):
            if 'order' in anno.__args__: 
                self.order=anno.__args__[-1]
        if self.order is None: 
            raise NodeException(f'Node: {self.name} needs Literal["order",int]')
        
    def __str__(self):  return self.name
    def __repr__(self): return str(self)
    def __lt__(self,o:'Node'): return self.order<o.order
    def __eq__(self,o:Union['Node',Callable]):
        return getattr(o,'function',o).__qualname__==self.function.__qualname__

    @property
    def name(self): return self.function.__name__
    @property
    def prefix(self): return self.name.split('_')[0]
    @property
    def postfix(self): return '_'.join(self.name.split('_')[1:])

    @classmethod
    def isvalid(cls,name)->bool:
        if isinstance(name,Callable): name=name.__name__
        return not name.startswith('_') and \
          any(name.startswith(pre) for pre in PREFIXES)

Nodes are self organizing parts of the Loop Graph.Their execution order is
determined by their relationship in the graph.

In [124]:
# export
class BaseLoop(object):
    
    @classmethod
    def nodes(cls,loops,instantiate=False):
        loops=L(loops)
        loop=cls() if instantiate else cls
        nodes=L(Node(node) for k,node in inspect.getmembers(loop) if Node.isvalid(k))
        nodes=nodes.sorted()
        for l in loops:
            print(l)
            for n in nodes:
                print(n)
                print(n in l.call_on)
            indexes=nodes.argwhere(lambda o:o in l.call_on)
            for i,idx in enumerate(indexes): 
                nodes.insert(idx+1,l.nodes(loops,instantiate))
                for j in range(i,len(indexes)): indexes[j]+=1
                
        return nodes

class Loop(BaseLoop):pass

class Outer(BaseLoop):
    def before_step(self) ->Literal['order',1]:  print('before_step')
    def on_step(self)     ->Literal['order',2]:  print('on_step')
    def after_step(self)  ->Literal['order',3]:  print('after_step')
    def failed_step(self) ->Literal['order',4]:  print('failed_step')
    def finally_step(self)->Literal['order',5]:  print('finally_step')
 
    def before_jump(self) ->Literal['order',6]:  print('before_jump')
    def on_jump(self)     ->Literal['order',7]:  print('on_jump')
    def after_jump(self)  ->Literal['order',8]:  print('after_jump')
    def failed_jump(self) ->Literal['order',9]:  print('failed_jump')
    def finally_jump(self)->Literal['order',10]: print('finally_jump')

class Inner(Loop):
    call_on=L(Outer.on_step,Outer.after_step,Outer.finally_jump)
    
    def before_iteration(self) ->Literal['order',1]: print('before_iteration')
    def on_iteration(self)     ->Literal['order',2]: print('on_iteration')
    def after_iteration(self)  ->Literal['order',3]: print('after_iteration')
    def failed_iteration(self) ->Literal['order',4]: print('failed_iteration')
    def finally_iteration(self)->Literal['order',5]: print('finally_iteration')
    
class FailingInner(Loop):
    call_on=L(Inner.finally_iteration)
    
    def on_force_fail(self) ->Literal['order',1]:                    
        print('on_force_fail')
        raise Exception

In [125]:
n=Outer.nodes([Inner,FailingInner])

<class '__main__.Inner'>
before_step
False
on_step
True
after_step
True
failed_step
False
finally_step
False
before_jump
False
on_jump
False
after_jump
False
failed_jump
False
finally_jump
True
<class '__main__.Inner'>
before_iteration
False
on_iteration
False
after_iteration
False
failed_iteration
False
finally_iteration
False
<class '__main__.FailingInner'>
before_iteration
False
on_iteration
False
after_iteration
False
failed_iteration
False
finally_iteration
True
<class '__main__.Inner'>
on_force_fail
False
<class '__main__.FailingInner'>
on_force_fail
False
<class '__main__.Inner'>
before_iteration
False
on_iteration
False
after_iteration
False
failed_iteration
False
finally_iteration
False
<class '__main__.FailingInner'>
before_iteration
False
on_iteration
False
after_iteration
False
failed_iteration
False
finally_iteration
True
<class '__main__.Inner'>
on_force_fail
False
<class '__main__.FailingInner'>
on_force_fail
False
<class '__main__.Inner'>
before_iteration
False
on_itera

TypeError: 'function' object is not iterable

In [119]:
list(n)

[before_step,
 on_step,
 (#5) [before_iteration,on_iteration,after_iteration,failed_iteration,finally_iteration],
 after_step,
 (#5) [before_iteration,on_iteration,after_iteration,failed_iteration,finally_iteration],
 failed_step,
 finally_step,
 before_jump,
 on_jump,
 after_jump,
 failed_jump,
 finally_jump,
 (#5) [before_iteration,on_iteration,after_iteration,failed_iteration,finally_iteration]]

In [13]:
out=inspect.signature(n[0].function)

In [17]:
out

<Signature (self) -> Literal['order', 1]>

In [49]:
n[0].function.__module__

'__main__'

In [51]:
test.__annotations__

{}

In [73]:
Inner.call_on[0].__qualname__

'Outer.on_step'

In [71]:
n[0].function.__qualname__

'Outer.before_step'

In [69]:
def run_or_return(f):
    try: return f()
    except: return f

{v:run_or_return(getattr(n[0].function,v)) for v in n[0].function.__dir__() if v not in ['__globals__']}

{'__repr__': '<function Outer.before_step at 0x7ff4919a4310>',
 '__call__': <method-wrapper '__call__' of function object at 0x7ff4919a4310>,
 '__get__': <method-wrapper '__get__' of function object at 0x7ff4919a4310>,
 '__new__': <function function.__new__(*args, **kwargs)>,
 '__closure__': None,
 '__doc__': None,
 '__module__': '__main__',
 '__code__': <code object before_step at 0x7ff491992f50, file "/tmp/ipykernel_107/810355912.py", line 13>,
 '__defaults__': None,
 '__kwdefaults__': None,
 '__annotations__': {'return': typing.Literal['order', 1]},
 '__dict__': {},
 '__name__': 'before_step',
 '__qualname__': 'Outer.before_step',
 '__hash__': 8793024472113,
 '__str__': '<function Outer.before_step at 0x7ff4919a4310>',
 '__getattribute__': <method-wrapper '__getattribute__' of function object at 0x7ff4919a4310>,
 '__setattr__': <method-wrapper '__setattr__' of function object at 0x7ff4919a4310>,
 '__delattr__': <method-wrapper '__delattr__' of function object at 0x7ff4919a4310>,
 '_

In [59]:
n[0].function.__class__

function

In [60]:
n[0].function.__class__

function

In [61]:
n[0].function.__class__

function

In [62]:
n[0].function.__class__

function

In [63]:
n[0].function.__class__

function

In [ ]:
n[0].function.__class__

In [58]:
n[0].function.__class__

function

In [26]:
n[0].function.__

function

In [ ]:
n.function

In [ ]:
class TestLoop:
    def on_iteration(self)->Literal['order',3]:pass
    def on_step(self)->Literal['order',3]:pass

In [ ]:
# hide
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.export import *
    from nbdev.export2html import *
    from nbverbose.cli import *
    make_readme()
    notebook2script()
    notebook2html()